In [ ]:
# Claude
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Actor-Critic Networks
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(ActorCritic, self).__init__()
        
        # Actor network (policy)
        self.actor = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, action_dim),
            nn.Softmax(dim=-1)
        )
        
        # Critic network (value function)
        self.critic = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
        
    def act(self, state):
        action_probs = self.actor(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        action_logprob = dist.log_prob(action)
        return action.detach(), action_logprob.detach()
    
    def evaluate(self, state, action):
        action_probs = self.actor(state)
        dist = Categorical(action_probs)
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy

# PPO Agent
class PPO:
    def __init__(
        self,
        state_dim,              # 8
        action_dim,             # 1
        lr_actor=0.0003,
        lr_critic=0.001,
        gamma=0.99,
        K_epochs=4,
        eps_clip=0.2,
        value_coef=0.5,
        entropy_coef=0.01
    ):
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        
        self.policy = ActorCritic(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam([
            {'params': self.policy.actor.parameters(), 'lr': lr_actor},
            {'params': self.policy.critic.parameters(), 'lr': lr_critic}
        ])
        
        self.policy_old = ActorCritic(state_dim, action_dim).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())       #copying the params of policy instance
        self.MseLoss = nn.MSELoss()
    
    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob = self.policy_old.act(state)
        
        return action.item(), action_logprob.item()
    
    def update(self, memory):
        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-8)
        
        # Convert lists to tensors
        old_states = torch.stack(memory.states).detach().to(device)
        old_actions = torch.tensor(memory.actions, dtype=torch.int64).detach().to(device)
        old_logprobs = torch.tensor(memory.logprobs, dtype=torch.float32).detach().to(device)
        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):
            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (π_θ / π_θ_old)
            ratios = torch.exp(logprobs - old_logprobs.detach())
            
            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            
            # Final loss of clipped objective PPO
            value_loss = self.MseLoss(state_values, rewards)
            policy_loss = -torch.min(surr1, surr2).mean()
            entropy_loss = -dist_entropy.mean()
            
            loss = policy_loss + self.value_coef * value_loss + self.entropy_coef * entropy_loss
            
            #Take gradient step
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

# Memory for storing transitions
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

# Training loop
def train():
    env_name = "LunarLander-v3"
    env = gym.make(env_name, continuous = False)
    
    state_dim = env.observation_space.shape[0] # 8
    action_dim = env.action_space.n # 1
    
    max_episodes = 5000        # max training episodes
    max_timesteps = 1500       # max timesteps in one episode
    update_timestep = 2000     # update policy every n timesteps
    save_interval = 500        # to save model every n episodes
    
    # PPO hyperparameters
    K_epochs = 8               # update policy for K epochs
    eps_clip = 0.2             # clip parameter for PPO
    gamma = 0.98               # discount factor
    lr_actor = 0.0003          # learning rate for actor
    lr_critic = 0.001          # learning rate for critic 
    
    # Initialize PPO agent
    ppo = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip)
    memory = Memory()
    
    # Logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0
    
    # Training loop
    for i_episode in range(1, max_episodes+1):
        state, _ = env.reset()
        
        for t in range(max_timesteps):
            timestep += 1
            
            # Select action from policy
            action, logprob = ppo.select_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Save in memory
            memory.states.append(torch.FloatTensor(state).to(device))
            memory.actions.append(action)
            memory.logprobs.append(logprob)
            memory.rewards.append(reward)
            memory.is_terminals.append(done)
            
            state = next_state
            
            # Update if its time
            if timestep % update_timestep == 0:
                ppo.update(memory)
                memory.clear_memory()
                timestep = 0
            
            running_reward += reward
            
            if done:
                break
                
        avg_length += t
        
        # Print average reward and length every 10 episodes
        if i_episode % 10 == 0:
            avg_length = avg_length/10
            running_reward = running_reward/10
            
            print(f'Episode {i_episode} \t Avg length: {avg_length:.2f} \t Reward: {running_reward:.2f}')
            running_reward = 0
            avg_length = 0
            
        # Save model
        if i_episode % save_interval == 0:
            torch.save(ppo.policy.state_dict(), f'./PPO_LunarLander_{i_episode}.pth')
            
    env.close()

if __name__ == '__main__':
    train()

In [ ]:
#Getting bad results w this implementation, Don't know why -> at least for nowwwww

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gymnasium as gym
from torch.distributions import Categorical

GAMMA = 0.99
GAE_LAMBDA = 0.95  
EPSILON = 0.2  # clipping range for PPO -> usually 0.1 or 0.2 rhta
ACTOR_LR = 3e-4
CRITIC_LR = 3e-4
BATCH_SIZE = 64
EPOCHS = 4
NUM_EPISODES = 1000
env = gym.make("LunarLander-v3", continuous=False)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

#Policy network as usual
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 128),  
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, state):
        return self.model(state)

#value function
class Critic(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def forward(self, state):
        return self.model(state)

actor = Actor(state_dim, action_dim)
critic = Critic(state_dim)
optimizer_actor = optim.Adam(actor.parameters(), lr=ACTOR_LR)
optimizer_critic = optim.Adam(critic.parameters(), lr=CRITIC_LR)


#Computing GAE
def compute_gae(rewards, values, gamma=GAMMA, lam=GAE_LAMBDA):
    """
    Iska explanation notes.ipynb me snippets dala hai
    and
    Here lambda is a hyperparameter, It can be multiplied with : 
    gamma and last_advantage for BIAS & VARIANCE Trade off
    """
    advantages = []
    last_advantage = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * values[t+1] - values[t]        #also known as one step TD Error : δt^V​=rt​+γV(st+1​)−V(st​)
        last_advantage = delta + gamma * lam * last_advantage       #A(t)GAE(γ,λ)​ = δt^V ​+ γ * λ * A(t+1)GAE(γ,λ)
        advantages.insert(0, last_advantage)
    advantages = torch.tensor(advantages, dtype=torch.float32)
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
    
    # Calculate : advantages = returns - values
    # Calculate : returns = advantages + values
    returns = advantages + torch.tensor(values[:-1], dtype=torch.float32)
    return advantages, returns

# Training loop
episode_rewards = []

for episode in range(NUM_EPISODES):
    if(episode > 900):
        env = gym.make("LunarLander-v3", continuous=False, render_mode='human')
    state, _ = env.reset()
    done = False
    episode_reward = 0
    
    log_probs = []
    values = []
    rewards = []
    states = []
    actions = []
    
    # Collect trajectory
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        action_probs = actor(state_tensor)
        value = critic(state_tensor).squeeze()
        
        dist = Categorical(action_probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        next_state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated
        
        states.append(state_tensor)
        actions.append(action)
        log_probs.append(log_prob)
        values.append(value.item()) 
        rewards.append(reward)
        
        state = next_state
        episode_reward += reward
    
    # appending final state value for GAE calculation
    with torch.no_grad():
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        final_value = critic(state_tensor).squeeze()
    values.append(final_value.item())
    
    # Compute advantages and returns
    advantages, returns = compute_gae(rewards, values)
    
    # Convert lists to tensors for batch processing
    # Adding detach to ensure no gradient flow
    old_states = torch.cat(states).detach()         #torch.Size([104, 8])
    old_actions = torch.stack(actions).detach()     #torch.Size([104, 1]) = [[n1], [n2], [n3],... [n_steps]]
    old_log_probs = torch.stack(log_probs).detach() #torch.Size([104, 1]) = [[n1], [n2], [n3],... [n_steps]]
    # print(old_actions)
   
    # PPO update
    for _ in range(EPOCHS):
        # Process in mini-batches
        indices = np.arange(len(old_states))
        np.random.shuffle(indices)
        
        for start_idx in range(0, len(old_states), BATCH_SIZE):
            end_idx = min(start_idx + BATCH_SIZE, len(old_states))
            batch_indices = indices[start_idx:end_idx]
            
            batch_states = old_states[batch_indices]
            batch_actions = old_actions[batch_indices]
            batch_log_probs = old_log_probs[batch_indices]
            batch_advantages = advantages[batch_indices]
            batch_returns = returns[batch_indices]
            
            # Actor update
            new_action_probs = actor(batch_states)
            new_dist = Categorical(new_action_probs)
            new_log_probs = new_dist.log_prob(batch_actions)
            
            # Calculate policy ratio and surrogate loss
            ratio = torch.exp(new_log_probs - batch_log_probs)
            surr1 = ratio * batch_advantages
            surr2 = torch.clamp(ratio, 1.0 - EPSILON, 1.0 + EPSILON) * batch_advantages
            
            # Negative because we're minimizing, but we want to maximize the objective
            entropy = new_dist.entropy().mean()
            actor_loss = -torch.min(surr1, surr2).mean() - 0.01 * entropy
            
            # Update actor
            optimizer_actor.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(actor.parameters(), 1)
            optimizer_actor.step()
            
            # Critic update - completely separate calculation
            critic_values = critic(batch_states).squeeze()
            critic_loss = nn.MSELoss()(critic_values, batch_returns)
            
            # Update critic
            optimizer_critic.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(critic.parameters(), 1)
            optimizer_critic.step()
            
    episode_rewards.append(episode_reward)
    if (episode + 1) % 10 == 0:
        avg_reward = np.mean(episode_rewards[-10:])
        print(f"Episode {episode+1}/{NUM_EPISODES}, Avg Reward (last 10): {avg_reward:.2f}")
env.close()

# Getting bad results ;-;